In [1]:
import glob
import skimage.io as io
import numpy as np
#import commonfunctions as cf
import sklearn
from skimage.feature import hog
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

import pickle
import cv2
import random
import os

from PIL import Image, ImageEnhance



from PIL import Image, ImageEnhance
import cv2
import matplotlib.pyplot as plt
import numpy as np


import argparse
import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.metrics import *
from sklearn.decomposition import PCA




/home/mohamed/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
'''
if you want to load the data again to work with them
'''

male_data=np.load('male_data_small.npy',allow_pickle=True)
female_data=np.load('female_data_small.npy',allow_pickle=True)

In [3]:
from skimage import feature, exposure

def HistogramofOrientedGradients(image):
    resized_img = resize(image, (128, 64))
    fd, hog_image = feature.hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True)
    
    return (np.array(fd)).flatten()



In [21]:
print(np.reshape(HistogramofOrientedGradients(male_data[0]),(1,-1)).shape)

temp=HistogramofOrientedGradients(male_data[0])
X=temp
X=np.reshape(X,(-1,1))
pca=PCA(n_components=10)
X=pca.fit_transform(X)
print(X)

(1, 3780)


ValueError: n_components=10 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [5]:


N_ANGLE_BINS = 40
BIN_SIZE = 360 // N_ANGLE_BINS
LEG_LENGTH = 25

def get_contour_pixels(bw_image):
    contours, _= cv2.findContours(
        bw_image, cv2.RETR_TREE, 
        cv2.CHAIN_APPROX_NONE
        ) 
    # contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[1:]
    
    img2 = bw_image.copy()[:,:,np.newaxis]
    img2 = np.concatenate([img2, img2, img2], axis = 2)
    
    # if self.show_images:
    #     for cnt in contours : 
    #         cv2.drawContours(img2, [cnt], 0, (255, 0, 0), 1)  
            
    #     plt.imshow(img2, cmap='gray')
    return contours

def get_hinge_features(bw_image):
    # if self.is_binary:
    #     bw_image, _ = self.preprocess_binary_image(img_file, self.sharpness_factor, self.bordersize)
    # else:
    #     bw_image, _ = self.preprocess_image(img_file, self.sharpness_factor, self.bordersize)
    
    contours = get_contour_pixels(bw_image)
    
    hist = np.zeros((N_ANGLE_BINS, N_ANGLE_BINS))
        
    # print([len(cnt) for cnt in contours])
    for cnt in contours:
        n_pixels = len(cnt)
        if n_pixels <= LEG_LENGTH:
            continue
        
        points = np.array([point[0] for point in cnt])
        xs, ys = points[:, 0], points[:, 1]
        point_1s = np.array([cnt[(i + LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        point_2s = np.array([cnt[(i - LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        x1s, y1s = point_1s[:, 0], point_1s[:, 1]
        x2s, y2s = point_2s[:, 0], point_2s[:, 1]
        
        phi_1s = np.degrees(np.arctan2(y1s - ys, x1s - xs) + np.pi)
        phi_2s = np.degrees(np.arctan2(y2s - ys, x2s - xs) + np.pi)
        
        indices = np.where(phi_2s > phi_1s)[0]
        
        for i in indices:
            phi1 = int(phi_1s[i] // BIN_SIZE) % N_ANGLE_BINS
            phi2 = int(phi_2s[i] // BIN_SIZE) % N_ANGLE_BINS
            hist[phi1, phi2] += 1
            
    normalised_hist = hist / np.sum(hist)
    feature_vector = normalised_hist[np.triu_indices_from(normalised_hist, k = 1)]
    
    return feature_vector


#Initialization:
N_RHO_BINS = 7
N_ANGLE_BINS = 12
N_BINS = N_RHO_BINS * N_ANGLE_BINS
BIN_SIZE = 360 // N_ANGLE_BINS
R_INNER = 5.0
R_OUTER = 35.0
K_S = np.arange(3, 8)
def get_cold_features(bw_image, approx_poly_factor = 0.01 , sharpness_factor = 10 , bordersize = 3):

    #bw_image, _ = preprocess_image(img_file, sharpness_factor, bordersize)
    contours = get_contour_pixels(bw_image)
    
    rho_bins_edges = np.log10(np.linspace(R_INNER, R_OUTER, N_RHO_BINS))
    feature_vectors = np.zeros((len(K_S), N_BINS))
    
    for j, k in enumerate(K_S):
        hist = np.zeros((N_RHO_BINS, N_ANGLE_BINS))
        for cnt in contours:
            epsilon = approx_poly_factor * cv2.arcLength(cnt,True)
            cnt = cv2.approxPolyDP(cnt,epsilon,True)
            n_pixels = len(cnt)
            
            point_1s = np.array([point[0] for point in cnt])
            x1s, y1s = point_1s[:, 0], point_1s[:, 1]
            point_2s = np.array([cnt[(i + k) % n_pixels][0] for i in range(n_pixels)])
            x2s, y2s = point_2s[:, 0], point_2s[:, 1]
            
            thetas = np.degrees(np.arctan2(y2s - y1s, x2s - x1s) + np.pi)
            rhos = np.sqrt((y2s - y1s) ** 2 + (x2s - x1s) ** 2)
            if(rhos.all()):
                rhos_log_space = np.log10(rhos)
            else:
                rhos_log_space = rhos
            
            quantized_rhos = np.zeros(rhos.shape, dtype=int)
            for i in range(N_RHO_BINS):
                quantized_rhos += (rhos_log_space < rho_bins_edges[i])
                
            for i, r_bin in enumerate(quantized_rhos):
                theta_bin = int(thetas[i] // BIN_SIZE) % N_ANGLE_BINS
                hist[r_bin - 1, theta_bin] += 1
            
        normalised_hist = hist / hist.sum()
        feature_vectors[j] = normalised_hist.flatten()
        
    return feature_vectors.flatten()

    

In [6]:
print(get_hinge_features(male_data[0]).shape)
print(male_data[0].shape)

(66,)
(1420, 1000)


In [7]:
print(get_cold_features(male_data[0]).shape)
print(get_hinge_features(male_data[0]).shape)

(420,)
(66,)


In [11]:
a=np.array([1,2,3])
b=np.array([4,5,6])
c=np.array([4,5,6])
d=np.concatenate((a,b,c),axis=0)
print(d)

[1 2 3 4 5 6 4 5 6]


In [45]:


male_feature=[]
female_feature=[]
# pca=PCA(n_components=70)
# X=pca.fit_transform(X)
# print(X.shape, Y.shape)

#print(get_hinge_features(male_data[0]).shape)
for i,img in enumerate(male_data):
    
    cold=get_cold_features(img)
    hinge=get_hinge_features(img)
    male_feature.append(np.concatenate((cold,hinge),axis=0))
    # print(i)

for i,img in enumerate(female_data):
    
    cold=get_cold_features(img)
    hinge=get_hinge_features(img)
    female_feature.append(np.concatenate((cold,hinge),axis=0))

male_feature=np.array(male_feature)
female_feature=np.array(female_feature)

In [52]:

X=np.concatenate((male_feature,female_feature),axis=0)

Y=np.concatenate((np.ones(male_feature.shape[0]),
                np.zeros(female_feature.shape[0])),
                axis=0)

In [53]:
pca=PCA(n_components=70)
X=pca.fit_transform(X)
print(X.shape, Y.shape)


(363, 70) (363,)


In [54]:
X,Y=sklearn.utils.shuffle(X,Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109) # 70% training and 30% test


In [55]:
clf = svm.SVC(kernel='rbf', verbose=True, C=10)

clf.fit(X_train, y_train)


[LibSVM]..*.

SVC(C=10, verbose=True)

*
optimization finished, #iter = 897
obj = -299.056826, rho = 0.741836
nSV = 194, nBSV = 6
Total nSV = 194


In [56]:

y_pred = clf.predict(X_train)

# print(y_pred)
# print(y_test)

In [57]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_train, y_pred))

Accuracy: 1.0


In [31]:
accuracy=0
X=np.concatenate((male_feature,female_feature),axis=0)
Y=np.concatenate((np.ones(male_feature.shape[0]),
                    np.zeros(female_feature.shape[0])),
                    axis=0)
pca=PCA(n_components=200)
X=pca.fit_transform(X)
for i in range(0,100):


    X,Y=sklearn.utils.shuffle(X,Y)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109) # 70% training and 30% test
    clf = svm.SVC(kernel='rbf', verbose=True, C=10)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    accuracy+=metrics.accuracy_score(y_test, y_pred)

    
    pass
print(accuracy/100)

[LibSVM].*.*
optimization finished, #iter = 656
obj = -210.652798, rho = 0.734826
nSV = 274, nBSV = 0
Total nSV = 274
[LibSVM].*.*
optimization finished, #iter = 694
obj = -218.420939, rho = 0.564902
nSV = 282, nBSV = 0
Total nSV = 282
[LibSVM].*.*
optimization finished, #iter = 654
obj = -210.314982, rho = 0.770618
nSV = 273, nBSV = 0
Total nSV = 273
[LibSVM].*.*
optimization finished, #iter = 703
obj = -219.611229, rho = 0.682399
nSV = 280, nBSV = 1
Total nSV = 280
[LibSVM].*.*
optimization finished, #iter = 671
obj = -219.110390, rho = 0.485715
nSV = 275, nBSV = 1
Total nSV = 275
[LibSVM].*.*
optimization finished, #iter = 679
obj = -219.404548, rho = 0.781577
nSV = 280, nBSV = 0
Total nSV = 280
[LibSVM].*.*
optimization finished, #iter = 679
obj = -212.938387, rho = 0.586280
nSV = 282, nBSV = 0
Total nSV = 282
[LibSVM].*.*
optimization finished, #iter = 662
obj = -208.966492, rho = 0.699143
nSV = 278, nBSV = 0
Total nSV = 278
[LibSVM].*.*
optimization finished, #iter = 707
obj = -2

In [34]:
print(male_feature.shape)

(232, 486)


In [ ]:
a=np.ones((3,3))
b=np.zeros((3,4))
print(a.shape,b.shape)
print(np.concatenate((a,b),axis=1))

(3, 3) (3, 4)
[[1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]]
